In [13]:
import numpy as np
import pandas as pd
import re
import scipy as sp
from scipy.sparse import hstack, vstack
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score
import lightgbm as lgb

import sklearn
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

import pickle
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_union

In [2]:
def save_varaible_pickle(var, name, model = False):
    filename = name + '.pickle'
    if model:
        joblib.dump(var, filename = filename)
    else:  
        with open(filename, 'wb') as f:
            pickle.dump(var, f, pickle.HIGHEST_PROTOCOL)

def load_variable_pickle(name, model = False):
    filename = name
    if model:
        return joblib.load(filename = filename)
    else:
        with open(filename, 'rb') as f:
            return pickle.load(f)
        

In [3]:
def choose_system(os):
    if os == 'mac':
        train_path = '/Users/yichen/Desktop/toxic/input/train.csv'
        test_path = '/Users/yichen/Desktop/toxic/input/test.csv'
    elif os == 'linux':
        train_path = '/home/yiche/Desktop/toxic/input/train.csv'
        test_path = '/home/yiche/Desktop/toxic/input/test.csv'
    elif os == 'aws':
        train_path = 'train.csv'
        test_path = 'test.csv'
    
    train = pd.read_csv(train_path).fillna(' ')
    test = pd.read_csv(test_path).fillna(' ')
    return train, test
    
    

In [4]:
def check_accuracy(true, estimate):
    acc = accuracy_score(true, estimate)
    logloss = log_loss(true, estimate)
    auc = roc_auc_score(true, esimate)
    print("accuracy score: {}, logloss: {}, auc: {}".format(acc, logloss, auc))

In [5]:
train_features.shape

NameError: name 'train_features' is not defined

In [6]:
train, test = choose_system('linux')
train_text = train['comment_text']
test_text = test['comment_text']
all_text = pd.concat([train_text, test_text])
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
test.shape

(153164, 2)

In [7]:
repl = {
    "&lt;3": " good ", ":d": " good ",
    ":dd": " good ",
    ":p": " good ",
    "8)": " good ",
    ":-)": " good ",
    ":)": " good ",
    ";)": " good ",
    "(-:": " good ",
    "(:": " good ",
    "yay!": " good ",
    "yay": " good ",
    "yaay": " good ",
    "yaaay": " good ",
    "yaaaay": " good ",
    "yaaaaay": " good ",
    ":/": " bad ",
    ":&gt;": " sad ",
    ":')": " sad ",
    ":-(": " bad ",
    ":(": " bad ",
    ":s": " bad ",
    ":-s": " bad ",
    "&lt;3": " heart ",
    ":d": " smile ",
    ":p": " smile ",
    ":dd": " smile ",
    "8)": " smile ",
    ":-)": " smile ",
    ":)": " smile ",
    ";)": " smile ",
    "(-:": " smile ",
    "(:": " smile ",
    ":/": " worry ",
    ":&gt;": " angry ",
    ":')": " sad ",
    ":-(": " sad ",
    ":(": " sad ",
    ":s": " sad ",
    ":-s": " sad ",
    r"\br\b": "are",
    r"\bu\b": "you",
    r"\bhaha\b": "ha",
    r"\bhahaha\b": "ha",
    r"\bdon't\b": "do not",
    r"\bdoesn't\b": "does not",
    r"\bdidn't\b": "did not",
    r"\bhasn't\b": "has not",
    r"\bhaven't\b": "have not",
    r"\bhadn't\b": "had not",
    r"\bwon't\b": "will not",
    r"\bwouldn't\b": "would not",
    r"\bcan't\b": "can not",
    r"\bcannot\b": "can not",
    r"\bi'm\b": "i am",
    "m": "am",
    "r": "are",
    "u": "you",
    "haha": "ha",
    "hahaha": "ha",
    "don't": "do not",
    "doesn't": "does not",
    "didn't": "did not",
    "hasn't": "has not",
    "haven't": "have not",
    "hadn't": "had not",
    "won't": "will not",
    "wouldn't": "would not",
    "can't": "can not",
    "cannot": "can not",
    "i'm": "i am",
    "m": "am",
    "i'll" : "i will",
    "its" : "it is",
    "it's" : "it is",
    "'s" : " is",
    "that's" : "that is",
    "weren't" : "were not",
}

In [8]:
keys = [i for i in repl.keys()]

new_train_data = []
new_test_data = []
ltr = train["comment_text"].tolist()
keys = [i for i in repl.keys()]

new_train_data = []
new_test_data = []
ltr = train["comment_text"].tolist()
lte = test["comment_text"].tolist()
for i in ltr:
    arr = str(i).split()
    xx = ""
    for j in arr:
        j = str(j).lower()
        if j[:4] == 'http' or j[:3] == 'www':
            continue
        if j in keys:
            # print("inn")
            j = repl[j]
        xx += j + " "
    new_train_data.append(xx)
for i in lte:
    arr = str(i).split()
    xx = ""
    for j in arr:
        j = str(j).lower()
        if j[:4] == 'http' or j[:3] == 'www':
            continue
        if j in keys:
            # print("inn")
            j = repl[j]
        xx += j + " "
    new_test_data.append(xx)
train["new_comment_text"] = new_train_data
test["new_comment_text"] = new_test_data

trate = train["new_comment_text"].tolist()
tete = test["new_comment_text"].tolist()
for i, c in enumerate(trate):
    trate[i] = re.sub('[^a-zA-Z ?!]+', '', str(trate[i]).lower())
for i, c in enumerate(tete):
    tete[i] = re.sub('[^a-zA-Z ?!]+', '', tete[i])
train["comment_text"] = trate
test["comment_text"] = tete
del trate, tete
train.drop(["new_comment_text"], axis=1, inplace=True)
test.drop(["new_comment_text"], axis=1, inplace=True)

train_text = train['comment_text']
test_text = test['comment_text']
all_text = pd.concat([train_text, test_text])

In [9]:
all_text.shape

(312735,)

In [9]:
word_vectorizer = TfidfVectorizer(
    strip_accents='unicode',
    analyzer='word',
    min_df = 2,
    stop_words = 'english',
    token_pattern=r'\w{1,}',
    ngram_range=(1, 3),
    max_features=30000
    )
    

char_vectorizer = TfidfVectorizer(
    strip_accents='unicode',
    analyzer='char',
    min_df = 2,
    ngram_range=(2, 6),
    max_features=30000)

vectorizer = make_union(word_vectorizer, char_vectorizer)

In [10]:
vectorizer.fit(all_text)
train_features = vectorizer.transform(train_text)
train_features.shape

/Users/yichen/anaconda/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


(159571, 60000)

In [11]:
test_features = vectorizer.transform(test_text)

/Users/yichen/anaconda/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [13]:
test_features.shape

(153164, 60000)

In [43]:
import pickle
from sklearn.externals import joblib
save_varible_pickle(train_features, train_features_save)
save_varible_pickle(test_features, test_features_save)
save_varible_pickle(clf, _logistic, model = True)

NameError: name 'train_features_save' is not defined

In [10]:
import pickle
from sklearn.externals import joblib
test_features = load_variable_pickle('test_features_save.pickle')
train_features = load_variable_pickle('train_features_save.pickle')
#clf = load_variable_pickle('_logistic_model5.pickle')

In [11]:
print(test_features.shape, train_features.shape)


(153164, 60000) (159571, 60000)


In [14]:
gbm = lgb.LGBMClassifier(objective = 'binary',
                        metric = 'binary_logloss',
                        num_leaves=31,
                        learning_rate=0.05,
                        n_estimators=20)
#gbm.fit(meta_train_toxic_prediction, meta_train_toxic_valid,
 #       eval_metric='auc')

log_model = LogisticRegression(solver='sag')
svc_model = LinearSVC(random_state=1023)

In [15]:
base_level_models = []
scores_auc = []
#submission = pd.DataFrame.from_dict({'id': test['id']})
y_fold_validation = load_variable_pickle('y_fold_validation.pickle')
#submission[class_name] = classifier.predict_proba(test_features)[:, 1]
MNB = MultinomialNB()

neigh2 = KNeighborsClassifier(n_neighbors=2)
neigh4 = KNeighborsClassifier(n_neighbors=4)
neigh8 = KNeighborsClassifier(n_neighbors=8)
neigh16 = KNeighborsClassifier(n_neighbors=16)
neigh32 = KNeighborsClassifier(n_neighbors=32)

etc_levelone = ExtraTreesClassifier(n_estimators= 50, random_state=11)
ada = AdaBoostClassifier(n_estimators=100)

gdc = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)


model_set = []
model_set.append(log_model)
model_set.append(svc_model)
model_set.append(etc_levelone)
model_set.append(ada)
model_set.append(MNB)
model_set.append(neigh2)
model_set.append(neigh4)
model_set.append(neigh8)
#model_set.append(neigh16)
#model_set.append(neigh32)
#base_level_models.append(log_model)
#base_level_models.append(svc_model)
len(base_level_models)

0

In [16]:
def ready_for_leveltwo(models, train_data, test_data, _train_target, meta_model):
    submission = pd.DataFrame.from_dict({'id': test['id']})
    for class_name in class_names:
        
        count = 0
        class_counter = 0
        train_target = train[class_name]
        for model in models:
            if count == 0:
                meta_train = cv_split(train = train_data, target=train_target, K = 5, random_state=1023, model = model)
            else:
                temp = cv_split(train = train_data, target=train_target, K = 5, random_state=1023, model = model)
                meta_train = np.column_stack((meta_train, temp))
            count = count + 1
        #print("cv finished")
        for model in models:
            model.fit(train_data, train_target)
            if count == len(models):
                meta_test = model.predict(test_data)
            else:
                temp = model.predict(test_data)
                meta_test = np.column_stack((meta_test, temp))
            count = count + 1
            
        
        meta_train_valid = y_fold_validation[:, class_counter]
        print("meta_train: {}, y_fold: {}, meta_test: {}".format(meta_train.shape, meta_train_valid.shape, meta_test.shape))
        meta_model.fit(meta_train, meta_train_valid) 
        a = meta_model.predict(meta_test)
        sub_name = 'submission' + str(class_name)
        save_varaible_pickle(a, sub_name)
        print(a.shape)
        submission[class_name] = a
        class_counter = class_counter + 1
        print("round: ")
    submission.to_csv('submission_final.csv', index=False)
    return submission

In [17]:
y_fold_validation[:, 0].shape

(159571,)

In [25]:
meta_train = ready_for_leveltwo(model_set, train_features, test_features, y_fold_validation, meta_model=gbm)
meta_train.shape

average roc_auc_score is: 0.8305854773252523  for model type: <class 'sklearn.linear_model.logistic.LogisticRegression'> 
average roc_auc_score is: 0.8579225490249766  for model type: <class 'sklearn.svm.classes.LinearSVC'> 
average roc_auc_score is: 0.7725433296764256  for model type: <class 'sklearn.ensemble.forest.ExtraTreesClassifier'> 
average roc_auc_score is: 0.8108591411860143  for model type: <class 'sklearn.ensemble.weight_boosting.AdaBoostClassifier'> 
average roc_auc_score is: 0.8615857737656055  for model type: <class 'sklearn.naive_bayes.MultinomialNB'> 


MemoryError: 

In [137]:
a.shape

(153164, 2)

In [32]:
train_toxic = train['toxic']
log_model = LogisticRegression(random_state=1023)
svc_model = LinearSVC(random_state=1023)
log_model.fit(train_features, train_toxic)
svc_model.fit(train_features, train_toxic)
log_train_prediction = log_model.predict(test_features)
svc_train_prediction = svc_model.predict(test_features)
print(log_train_prediction.shape, svc_train_prediction.shape)

(153164,) (153164,)


In [38]:
log_train_prediction

array([1, 0, 0, ..., 0, 0, 1])

In [49]:
meta_train_valid = load_variable_pickle('y_fold_validation.pickle')
meta_train_toxic_valid = meta_train_valid[:, 0]

In [24]:
def cv_split(train, target, K, random_state ,model):
    acc = []
    row = train.shape[0]
    if row != target.shape[0]:
        raise ValueError("At least one array required as input")
    num_per_fold = np.int(np.ceil(train.shape[0]/K))
    x_train, x_fold, y_train, y_fold1 = train_test_split(train, target, test_size = num_per_fold, random_state = random_state)
    model.fit(x_train, y_train)
    pred = model.predict(x_fold)
    acc.append(roc_auc_score(y_fold1, pred))
    #print("accuracy is {}".format(accuracy_score(y_fold1, pred)))
    i = 1
    if train.size != row:
        while i < K:
            if i != K - 1:
                x_fold = train[num_per_fold * i: num_per_fold * (i + 1), :]
                y_fold = target[num_per_fold * i: num_per_fold * (i + 1)]
                x_train = vstack([train[0: num_per_fold * i, : ], train[num_per_fold * (i + 1): , :]])
                y_train = pd.concat([target[0 : num_per_fold * i], target[num_per_fold * (i + 1): ]])
                #print("x_train: {}, y_train: {}, x_fold: {}, y_fold: {}".format(x_train.shape, y_train.shape, x_fold.shape, y_fold.shape))
            elif i == K-1:
                x_fold = train[num_per_fold * i: , :]
                y_fold = target[num_per_fold * i: ]
                x_train = train[0:num_per_fold * i, :]
                y_train = target[0:num_per_fold * i]
                
            
            y_fold1 = pd.concat([y_fold1[:], y_fold[:]])
            model.fit(x_train, y_train)
            pred1 = model.predict(x_fold)
            acc.append(roc_auc_score(y_fold, pred1))
            #rint("accuracy is {}".format(accuracy_score(y_fold, pred1)))
            pred = np.concatenate((pred, pred1), axis = 0)
            i = i + 1
    print("average roc_auc_score is: {}  for model type: {} ".format(np.mean(acc), type(model)))
    return pred# pred_all_testep

In [35]:
svc_fold_model = LinearSVC(random_state=1023)
log_fold_model = LogisticRegression(random_state=1023)
svc_meta_train_pred = cv_split(train=train_features, target=train_toxic, K = 5, random_state=1023, model=svc_fold_model)
log_meta_train_pred = cv_split(train=train_features, target=train_toxic, K = 5, random_state=1023, model=log_fold_model)

accuracy is 0.9616794610684631
accuracy is 0.9613661287795707
accuracy is 0.9627761240795863
accuracy is 0.9618674604417985
accuracy is 0.9620820406756291
accuracy is 0.9612094626351245
accuracy is 0.9614914616951277
accuracy is 0.9625254582484725
accuracy is 0.9615227949240169
accuracy is 0.9629594810566889


In [36]:
print(log_meta_train_pred.shape, svc_meta_train_pred.shape)

(159571,) (159571,)


In [37]:
log_meta_train_pred

array([0, 0, 0, ..., 0, 0, 0])

In [50]:
meta_train_toxic_prediction = np.column_stack((svc_meta_train_pred, log_meta_train_pred))
meta_test_toxic_prediction = np.column_stack((svc_train_prediction, log_train_prediction))
print(meta_train_prediction.shape, meta_test_prediction.shape)

(159571, 2) (153164, 2)


In [51]:
#param = {'num_leaves':31, 'num_trees':100, 'objective':'binary'}
#param['metric'] = 'auc'
#bst = lgb.train(param, train_data, num_round, valid_sets=[test_data])
gbm = lgb.LGBMRegressor(objective='binary',
                        num_leaves=31,
                        learning_rate=0.05,
                        n_estimators=20)
gbm.fit(meta_train_toxic_prediction, meta_train_toxic_valid,
        eval_metric='auc')

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       learning_rate=0.05, max_depth=-1, min_child_samples=20,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=20,
       n_jobs=-1, num_leaves=31, objective='binary', random_state=None,
       reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
       subsample_for_bin=200000, subsample_freq=1)

In [123]:
meta_train_toxic_valid.shape

(159571,)

In [56]:
gbm_toxic_pred_level2 = gbm.predict(meta_test_toxic_prediction)
gbm_toxic_pred_level2.shape

(153164,)

In [52]:
def single_class_cv_mutiple_time(model):
    result = np.arange(159571*6).reshape(159571,6)
    #all_test_result = np.arange(153164*6).reshape(153164,6)
    col = 0
    for class_name in class_names:
        train_target = train[class_name]
        test = test_features
        pred1, overall_test_result= cv_split(train = train_features, target = train_target, K = 5, random_state=1023, model=model)
        result[:, col] = pred1
        all_test_result[:, col] = overall_test_result
        #y_fold_result[:, col] = y_fold
        col = col + 1
        print("pred shape: {} and overall_test shape: {} ".format(pred1.shape, all_test_result.shape))
    
    return result, all_test_result


In [57]:
from sklearn.naive_bayes import MultinomialNB
MNB = MultinomialNB()
mnb_pred, mnb_pred_test= single_class_cv_mutiple_time(MNB)
save_varible_pickle(mnb_pred, 'mnb_pred_levelone')
save_varible_pickle(mnb_pred_test, 'mnb_pred_test_levelone')
mnb_pred.shape

accuracy is 0.9428795237349209
accuracy is 0.943944853517155
accuracy is 0.9435061883127056
accuracy is 0.9416261945793514
accuracy is 0.9429350380746451
pred shape: (159571,) and overall_test shape: (153164, 6) 
accuracy is 0.9858687137709541
accuracy is 0.9857747140842864
accuracy is 0.9851167162776124
accuracy is 0.984928716904277
accuracy is 0.9860863025289085
pred shape: (159571,) and overall_test shape: (153164, 6) 
accuracy is 0.9665674447751841
accuracy is 0.9663794454018486
accuracy is 0.9668807770640765
accuracy is 0.9658154472818424
accuracy is 0.9660305223903983
pred shape: (159571,) and overall_test shape: (153164, 6) 
accuracy is 0.9960833463888453
accuracy is 0.9956133479555068
accuracy is 0.9962713457621808
accuracy is 0.9962086793044023
accuracy is 0.9962708783804958
pred shape: (159571,) and overall_test shape: (153164, 6) 
accuracy is 0.9642174526084913
accuracy is 0.9624941250195833
accuracy is 0.9626194579351403
accuracy is 0.9629954566818111
accuracy is 0.96230140

(159571, 6)

In [58]:
from sklearn.ensemble import RandomForestClassifier
rfc_levelone = RandomForestClassifier(n_estimators= 50, random_state=11)
rfc_pred, rfc_predtest = single_class_cv_mutiple_time(rfc_levelone)
save_varible_pickle(rfc_pred, 'rfc_pred_levelone')
save_varible_pickle(rfc_pred_test, 'rfc_pred_test_levelone')
rfc_pred.shape

accuracy is 0.9533761554128153
accuracy is 0.9542221525928247
accuracy is 0.954660817797274
accuracy is 0.9545981513394955
accuracy is 0.9535896712732287
pred shape: (159571,) and overall_test shape: (153164, 6) 
accuracy is 0.9897853673821088
accuracy is 0.9906940310198966
accuracy is 0.9895973680087733
accuracy is 0.9898793670687764
accuracy is 0.9903168186518755
pred shape: (159571,) and overall_test shape: (153164, 6) 
accuracy is 0.9798527338242206
accuracy is 0.978975403415322
accuracy is 0.9800093999686668
accuracy is 0.9805107316308945
accuracy is 0.9795681739838927
pred shape: (159571,) and overall_test shape: (153164, 6) 
accuracy is 0.997086009713301
accuracy is 0.9965533448221839
accuracy is 0.9973366755444149
accuracy is 0.9976500078333073
accuracy is 0.9972109930744885
pred shape: (159571,) and overall_test shape: (153164, 6) 
accuracy is 0.9709540968196773
accuracy is 0.9712674291085697
accuracy is 0.9698887670374432
accuracy is 0.9729280902396992
accuracy is 0.969947666

(159571, 6)

In [59]:
from sklearn.ensemble import ExtraTreesClassifier
etc_levelone = ExtraTreesClassifier(n_estimators= 50, random_state=11)
etc_pred, etc_pred_test = single_class_cv_mutiple_time(etc_levelone)
save_varible_pickle(etc_pred, 'etc_pred_levelone')
save_varible_pickle(etc_pred_test, 'etc_pred_test_levelone')
etc_pred.shape

KeyboardInterrupt: 

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
neigh2 = KNeighborsClassifier(n_neighbors=2)
neigh2_pred, neigh2_pred_test= single_class_cv_mutiple_time(neigh2)
save_varible_pickle(neigh2_pred, 'neigh2_pred_levelone')
save_varible_pickle(neigh2_pred_test, 'neigh2_pred_test_levelone')
neigh2_pred.shape

In [ ]:
neigh4 = KNeighborsClassifier(n_neighbors=4)
neigh4_pred, neigh4_pred_test= single_class_cv_mutiple_time(neigh4)
save_varible_pickle(neigh4_pred, 'neigh4_pred_levelone')
save_varible_pickle(neigh4_pred_test, 'neigh4_pred_test_levelone')
neigh4_pred.shape

In [ ]:
neigh8 = KNeighborsClassifier(n_neighbors=8)
neigh8_pred = single_class_cv_mutiple_time(neigh8)
neigh8_pred.shape
save_varible_pickle(neigh8_pred, 'neigh8_pred_levelone')

In [ ]:
neigh32 = KNeighborsClassifier(n_neighbors=32)
neigh32_pred = single_class_cv_mutiple_time(neigh32)
neigh32_pred.shape
save_varible_pickle(neigh32_pred, 'neigh32_pred_levelone')

In [ ]:
neigh16 = KNeighborsClassifier(n_neighbors=16)
neigh16_pred = single_class_cv_mutiple_time(neigh16)
neigh16_pred.shape
save_varible_pickle(neigh16_pred, 'neigh16_pred_levelone')

In [ ]:
neigh64 = KNeighborsClassifier(n_neighbors=64)
neigh64_pred = single_class_cv_mutiple_time(neigh64)
neigh64_pred.shape
save_varible_pickle(neigh64_pred, 'neigh64_pred_levelone')

In [ ]:
neigh128 = KNeighborsClassifier(n_neighbors=128)
neigh128_pred = single_class_cv_mutiple_time(neigh128)
neigh128_pred.shape
save_varible_pickle(neigh128_pred, 'neigh128_pred_levelone')

In [ ]:
neigh256 = KNeighborsClassifier(n_neighbors=256)
neigh256_pred = single_class_cv_mutiple_time(neigh256)
neigh256_pred.shape
save_varible_pickle(neigh256_pred, 'neigh256_pred_levelone')

In [98]:
print(svc_pred.shape, log_pred.shape, mnb_pred.shape, rfc_pred.shape)

(159571, 6) (159571, 6) (159571, 6) (159571, 6)


In [102]:
y_fold_validation.shape

(159571, 6)

In [104]:
train.shape

(159571, 8)

In [105]:
stacked_prediction = np.column_stack((svc_pred, log_pred, mnb_pred, rfc_pred))
stacked_prediction

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [86]:
stacked_test_prediction = np.column_stack((svc_pred_test, log_pred_test, mnb_pred_test, rfc_pred_test))
stacked_test_prediction.shape

(153164, 24)

In [87]:
y_fold_validation.shape

(159571, 6)

In [88]:
y = np.column_stack((y_fold_validation, y_fold_validation, y_fold_validation, y_fold_validation))

In [89]:
y.shape

(159571, 24)

In [73]:
from sklearn.neighbors import KNeighborsClassifier
meta_model = KNeighborsClassifier(n_neighbors=3)

In [90]:
meta_model.fit(stacked_prediction, y_fold_validation)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform')

In [97]:
submission = meta_model.predict_proba(test_features)

ValueError: kd_tree does not work with sparse matrices. Densify the data, or set algorithm='brute'

In [78]:
submission = meta_model.predict_proba(test_features)

ValueError: kd_tree does not work with sparse matrices. Densify the data, or set algorithm='brute'

In [75]:
submission = meta_model.predict_proba(stacked_test_prediction)
submission.shape

AttributeError: 'list' object has no attribute 'shape'

In [84]:
submission

[array([[ 0.,  1.],
        [ 1.,  0.],
        [ 1.,  0.],
        ..., 
        [ 1.,  0.],
        [ 1.,  0.],
        [ 0.,  1.]]), array([[ 0.66666667,  0.33333333],
        [ 1.        ,  0.        ],
        [ 1.        ,  0.        ],
        ..., 
        [ 1.        ,  0.        ],
        [ 1.        ,  0.        ],
        [ 1.        ,  0.        ]]), array([[ 0.,  1.],
        [ 1.,  0.],
        [ 1.,  0.],
        ..., 
        [ 1.,  0.],
        [ 1.,  0.],
        [ 0.,  1.]]), array([[ 1.,  0.],
        [ 1.,  0.],
        [ 1.,  0.],
        ..., 
        [ 1.,  0.],
        [ 1.,  0.],
        [ 1.,  0.]]), array([[ 0.33333333,  0.66666667],
        [ 1.        ,  0.        ],
        [ 1.        ,  0.        ],
        ..., 
        [ 1.        ,  0.        ],
        [ 1.        ,  0.        ],
        [ 1.        ,  0.        ]]), array([[ 0.66666667,  0.33333333],
        [ 1.        ,  0.        ],
        [ 1.        ,  0.        ],
        ..., 
        [ 1

In [ ]:
submission = pd.DataFrame.from_dict({'id': test['id']})
for class_name in class_names:
    submission[class_name] = classifier.predict_proba(test_features)[:, 1]

In [62]:
def leveltwo_meta_model():
    meta_model = LinearSVC()
    stacked_prediction = np.column_stack((svc_pred, log_pred, mnb_pred, rfc_pred))
    stacked_test_prediction = np.column_stack((svc_pred_test, log_pred_test, mnb_pred_test, rfc_pred_test))
    y_valid = y_fold_validation
    meta_model.fit(stacked_prediction, y_valid)
    sub = meta_model.pred(stacked_test_prediction)
    return sub

In [63]:
submission = leveltwo_meta_model()

ValueError: bad input shape (159571, 6)

In [18]:
K = 5
num_per_fold = np.int(np.ceil(train_features.shape[0]/K))

In [143]:
y_fold_validation.shape
test_features.shape

(153164, 60000)

In [139]:
scores_auc = []
submission = pd.DataFrame.from_dict({'id': test['id']})
#classifier = LogisticRegression()
#for class_name in class_names:
train_target = train['toxic']
    #print(train_target.shape)
classifier = LogisticRegression(solver='sag')

    #cv_score = np.mean(cross_val_score(
     #   classifier, train_features, train_target, cv=3, scoring='neg_log_loss'))
    #scores_auc.append(np.abs(cv_score))
    #print('auc CV score for class {} is {}'.format(class_name, np.abs(cv_score)))

classifier.fit(train_features, train_target)
a = classifier.predict_proba(test_features)
print(a.shape)
a
submission[class_name] = a[:, 1]
    #save_varible_pickle(classifier, _logistic, model = True)
#print('Total auc CV score is {}'.format(np.mean(scores_auc)))

(153164, 2)


In [18]:
submission.to_csv('submission.csv', index=False)